In [1]:
#Import libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

#Create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#Import the GL Bot corpus file for preprocessing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
firstaid_file = open("firstaid.json").read()
intents = json.loads(firstaid_file)

### Data Understanding

In [2]:
intents

{'intents': [{'tag': 'Greetings',
   'patterns': ['hi',
    'how are you',
    'is anyone there',
    'hello',
    'whats up',
    'hey',
    'yo',
    'listen',
    'please help me',
    'i am learner from',
    'i belong to',
    'first aid',
    'i need help',
    'sos',
    'help',
    'hi chatbot',
    'hi',
    'chatbot',
    'hey ya',
    'how do I start',
    'talking to you for first time',
    'learn first aid'],
   'responses': ['Hello ^_^! How can I help you?'],
   'context_set': ''},
  {'tag': 'Exit',
   'patterns': ['thank you',
    'thanks',
    'cya',
    'see you',
    'later',
    'see you later',
    'goodbye',
    'i am leaving',
    'have a Good day',
    'you helped me',
    'thanks a lot',
    'thanks a ton',
    'you are the best',
    'great help',
    'you saved me',
    'you are a good learning buddy'],
   'responses': ['I hope I was able to assist you, good bye! ^_^'],
   'context_set': ''},
  {'tag': 'Emergency Number',
   'patterns': ['Emergency number',
 

### Data Pre-processing

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #Add documents in the corpus
        documents.append((w, intent['tag']))

        #Add into classes of list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yongg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yongg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#Lemmatize, lowercase each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#Sort the words by classes
classes = sorted(list(set(classes)))

#Documents = combination between patterns and intents
print (len(documents), "documents")

#Classes = intents
print (len(classes), "classes", classes)

#Words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

#Creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

254 documents
46 classes ['Abdominal Pain', 'Abrasions', 'Animal Bite', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Emergency Number', 'Exit', 'Eye Injury', 'Fainting', 'Fever', 'Food Poison', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Greetings', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bite', 'Nasal Congestion', 'Normal Bleeding', 'Nose Bleed', 'Pulled Muscle', 'Rash', 'Rectal Bleeding', 'Seizure', 'Skin problems', 'Snake bite', 'Sore Throat', 'Splinter', 'Sprains', 'Stings', 'Strains', 'Sun Burn', 'Teeth', 'Vertigo', 'Wound']
182 unique lemmatized words ['a', 'abdominal', 'abdominalpain', 'abrasion', 'aid', 'allergy', 'am', 'ambulance', 'an', 'animal', 'anyone', 'apply', 'are', 'belong', 'best', 'better', 'bit', 'bite', 'bitten', 'bleed', 'bleeding', 'blocked', 'blood', 'bring', 'broken', 'bruise', 'buddy', 'burn', 'by', 'call', 'cat', 'cause', 'ccts', 'chatbot', 'chemical', 'c

In [5]:
#Create the training data
training = []

#Create an empty array for the output
output_empty = [0] * len(classes)

#Training set, bag of words for each sentence
for doc in documents:
    
    #Initialize our bag of words
    bag = []
    
    #List of tokenized words for the pattern
    pattern_words = doc[0]
   
    #Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    #Create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    #Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#Shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

#Create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data has been created")

Training data has been created


### Modeling

In [6]:
#Create Neural Network model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=10, verbose=1)
model.save('chatbot.h5', hist) #We will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/300
26/26 [==============================] - 0s 907us/step - loss: 3.8480 - accuracy: 0.0472
Epoch 2/300
26/26 [==============================] - 0s 801us/step - loss: 3.8027 - accuracy: 0.1063
Epoch 3/300
26/26 [==============================] - 0s 800us/step - loss: 3.7517 - accuracy: 0.1142
Epoch 4/300
26/26 [==============================] - 0s 761us/step - loss: 3.7170 - accuracy: 0.0984
Epoch 5/300
26/26 [==============================] - 0s 833us/step - loss: 3.6568 - accuracy: 0.0866
Epoch 6/300
26/26 [==============================] - 0s 800us/step - loss: 3.6471 - accuracy: 0.0906
Epoch 7/300
26/26 [==============================] - 0s 840us/step - loss: 3.6015 - accuracy: 0.1024
Epoch 8/300
26/26 [==============================] - 0s 800us/step - loss: 3.5013 - accuracy: 0.1024
Epoch 9/300
26/26 [==============================] - 0s 760us/step - loss: 3.4624 - accuracy: 0.1220
Epoch 10/300
26/26 [==============================] - 0s 800us/step - loss: 3.4003 - accura

In [7]:
#Load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("firstaid.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

### Deployment

In [8]:
def clean_up_sentence(sentence):

    #Tokenize the pattern > split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    #Stem each word > create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


#Return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):

    #Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    #Bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # Assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    #Filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    #Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [9]:
#Function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [103]:
#Fucntion to rate the service before end conversation
def rateService():
    print("-"*50)
    print("\n"+ "Thank you for using our AI chatbot. It would be great if you could rate Sophie chatbot." + "\n")
    print("1 star  - Very unsatisfied")
    print("2 stars - Unsatisfied")
    print("3 stars - Neutral")
    print("4 stars - Satisdfied")
    print("5 stars - Very satisfied")
    
    rate = int(input("Your rating (1-5) > "))
    while rate <= 0 or rate > 5:
        print("Invalid! Please enter 1 to 5 only.")
        rate = int(input("Your rating (1-5) > "))
    if rate == 1 or rate == 2:
            print("Sorry for the bad experience :( Let's hear us your feedback, it is valuable to us.")
            feedback = str(input("Feedback: "))
            print("Thank you for your feedback. We will use your feedback for future improvement.")
    else:
            print("We really appreciate you taking the time to share your rating with us, thank you! :)")
   
    with open('RATE.txt', 'a') as f:
        f.write('%d' % rate)
        f.write('|')
        if rate == 1 or rate == 2:
                f.write(feedback)
                f.write('|')
        if rate == 3:
                f.write('Neutral')
                f.write('|')
        if rate == 4:
                f.write('Good Service')
                f.write('|')
        else:
                f.write('Excellent Service')
                f.write('|')       

In [107]:
#Function to start the chat bot which will continue until the user type 'end' or 'bye'

def startChat():
    print("Shopie Bot: Hello! I am Sophie, your personal AI first aid assistant.")
    name = str(input("Sophie Bot: What is your name ? > "))
    print("Sophie Bot: Nice to meet you, " + (name) + "!")
    print("-"*50)
    while True:
        inp = str(input((name)+": "))
        if inp.lower()=="end" or inp.lower() =="bye":
            print("Sophie Bot: Bye " + (name) + "!")
            rateService()
            break
        if inp.lower()== '' or inp.lower()== '*': #If user empty input
            print('Please enter again!')
            print("-"*50)
        else:
            print(f"Sophie Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)
    
    
    with open('RATE.txt', 'a') as f:
        f.write(name)
        f.write("\n")       
        

In [110]:
#Start the chat bot

startChat()

Shopie Bot: Hello! I am Sophie, your personal AI first aid assistant.
Sophie Bot: What is your name ? > Kimno
Sophie Bot: Nice to meet you, Kimno!
--------------------------------------------------
Kimno: yea
1/1 [==============================] - 0s 13ms/step
Sophie Bot: Hello ^_^! How can I help you?

--------------------------------------------------
Kimno: no
1/1 [==============================] - 0s 13ms/step
Sophie Bot: Hello ^_^! How can I help you?

--------------------------------------------------
Kimno: bye
Sophie Bot: Bye Kimno!
--------------------------------------------------

Thank you for using our AI chatbot. It would be great if you could rate Sophie chatbot.

1 star  - Very unsatisfied
2 stars - Unsatisfied
3 stars - Neutral
4 stars - Satisdfied
5 stars - Very satisfied
Your rating (1-5) > 3
We really appreciate you taking the time to share your rating with us, thank you! :)
